## HWW5:

В домашньому завданні до даного модулю ви потренуєтесь робити тестове завдання для влаштування на роботу. 
За даними акселерометра з мобільного телефону потрібно класифікувати, якою діяльністю займається людина: йде, стоїть, біжить чи йде по сходах. 
Знайти датасет ви можете за посиланням....

Використайте алгоритми SVM та випадковий ліс з бібліотеки scikit-learn. Як характеристики можете брати показники з акселерометра, проте щоб покращити результати роботи алгоритмів, спочатку можна підготувати наш датасет і розрахувати часові ознаки (time domain features). Більше ці характеристики описані в даній статті.

Порівняйте результати роботи обох алгоритмів на різних фічах та різні моделі між собою.

In [134]:
import math as m
from pathlib import Path
import pickle
from typing import Union

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error
%matplotlib inline

## Data preparation:

In [135]:
def read_from_csvfile(file: Path) -> pd.DataFrame:
    """Read content from csv-file and return dataframe from content."""
    df = pd.read_csv(file)
    
    return df

In [148]:
df = pd.DataFrame()
classification_human_activity = {}

- https://www.geeksforgeeks.org/different-ways-to-create-pandas-dataframe/
- https://pandas.pydata.org/docs/reference/api/pandas.concat.html
- https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.std.html
- https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.skew.html
- https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.kurt.html
- https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.var.html
- https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.idxmax.html
- https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.idxmin.html
- https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.entropy.html
- https://stackoverflow.com/questions/53177327/how-to-compute-shannon-entropy-of-information-from-a-pandas-dataframe
- https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html
- https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_error.html

In [149]:
def get_statistical_features(y: str, df: pd.DataFrame) -> pd.DataFrame:
    # y - type real data; df - sample data. 
    if y not in classification_human_activity:
        classification_human_activity[y] = len(classification_human_activity)

    data = {
            'activity': [classification_human_activity[y]],
            # 'max_x': [max(df['accelerometer_X'])],  # [df['accelerometer_X'].describe().loc['max']],
            # 'min_x': [min(df['accelerometer_X'])],  # [df['accelerometer_X'].describe().loc['min']],
            # 'mean_x': [df['accelerometer_X'].mean(axis=0)],  # [df['accelerometer_X'].describe().loc['mean']],
            # 'med_x': [df['accelerometer_X'].median(axis=0)],  # median
            # 'std_x': [df['accelerometer_X'].std(axis=0)],  # Standard deviation !!= [df['accelerometer_Y'].describe().loc['std']], 
            # 'skew_x': [df['accelerometer_X'].skew(axis=0)],  # skewness
            # 'kurt_x': [df['accelerometer_X'].skew(axis=0)],  # kurtosis
            # 'var_x': [df['accelerometer_X'].var(axis=0)],  # variance
            # 'idxmax_x': [df['accelerometer_X'].idxmax(axis=0)],  # index of first occurrence of maximum over requested axis
            # 'idxmin_x': [df['accelerometer_X'].idxmin(axis=0)],  # index of first occurrence of minimum over requested axis
            }
    [data.update({
                  f'max_{col[-1]}':[max(df[col])],
                  f'min_{col[-1]}':[min(df[col])],
                  f'mean_{col[-1]}':[df[col].mean(axis=0)],
                  f'med_{col[-1]}':[df[col].median(axis=0)],  # median
                  f'std_{col[-1]}':[df[col].std(axis=0)],  # Standard deviation
                  f'skew_{col[-1]}':[df[col].skew(axis=0)],  # skewness
                  f'kurt_{col[-1]}':[df[col].kurt(axis=0)],  # kurtosis
                  f'var_{col[-1]}':[df[col].var(axis=0)],  # variance
                  f'idxmax_{col[-1]}':[df[col].idxmax(axis=0)],  # index of first occurrence of maximum over requested axis
                  f'idxmin_{col[-1]}':[df[col].idxmin(axis=0)],  # index of first occurrence of minimum over requested axis
                #   f'rmse_{col[-1]}':[mean_squared_error(df[col], np.array([df[col].mean(axis=0) for _ in range(df.shape[0])]), squared=False)],  # Root Mean Square Error
                #   f'mae_{col[-1]}':[mean_absolute_error(df[col], np.array([df[col].mean(axis=0) for _ in range(df.shape[0])]))],  # mean absolute error
                  }) 
        for col in df.columns]

    [data.update({
                #   f'rmse_{col[-1]}':[mean_squared_error(df[col], np.array([data[f'mean_{col[-1]}'] for __ in range(df.shape[0])]), squared=False)],  # variance# Root Mean Square Error
                #   f'mae_{col[-1]}':[mean_absolute_error(df[col], np.array([data[f'mean_{col[-1]}'] for __ in range(df.shape[0])]))],  # mean absolute error
                  f'rmse_{col[-1]}':[mean_squared_error(df[col], [data[f'mean_{col[-1]}'] for __ in range(df.shape[0])], squared=False)],  # variance# Root Mean Square Error
                  f'mae_{col[-1]}':[mean_absolute_error(df[col], [data[f'mean_{col[-1]}'] for __ in range(df.shape[0])])],  # mean absolute error
                  }) 
        for col in df.columns]
    df_f = pd.DataFrame(data)
    
    return df_f

In [138]:
def get_data_to_dafaframe(path: Union[str, Path], df: pd.DataFrame) -> pd.DataFrame:
    path = Path(path)
    if not path.is_dir() or not path.exists():
        return None
    
    for file_system_object in path.iterdir():
        if file_system_object.is_dir():
            df = get_data_to_dafaframe(file_system_object, df)
            
        elif file_system_object.suffix.lower() in ('.csv',):
            df_add = read_from_csvfile(file_system_object)
            df = pd.concat([df, get_statistical_features(file_system_object.stem.split('-')[0], df_add)], ignore_index=True)

    return df

In [139]:
def save_prepared_data(df: pd.DataFrame, file_name: str='data1.bin') -> None:
    with open(file_name, 'wb') as fh:
        pickle.dump(df, fh)


In [140]:
def load_prepared_data(file_name: str='data1.bin') -> pd.DataFrame:
    with open(file_name, 'rb') as fh:
        df = pickle.load(fh)

    return df

In [150]:
if Path('data0.bin').is_file() and Path('data1.bin').is_file():
    df = load_prepared_data()
    classification_human_activity = load_prepared_data('data0.bin')

else:
    df = get_data_to_dafaframe('data', df)
    save_prepared_data(df)
    save_prepared_data(classification_human_activity, 'data0.bin')

In [151]:
df.tail(6)

,activity,max_X,min_X,mean_X,med_X,std_X,skew_X,kurt_X,var_X,idxmax_X,...,kurt_Z,var_Z,idxmax_Z,idxmin_Z,rmse_X,mae_X,rmse_Y,mae_Y,rmse_Z,mae_Z
6456,3,1.024718,-10.036493,-3.997519,-3.380613,3.011451,-0.338673,-0.557672,9.068838,5,...,0.374707,6.062126,26,21,2.960835,2.408620,4.719099,3.860922,2.420755,1.881598
6457,3,8.398860,-6.224924,0.826638,1.244985,2.653484,-0.012359,2.275576,7.040978,8,...,9.031841,30.027026,7,11,2.608884,1.942251,3.153400,2.494843,5.387590,3.226745
6458,3,12.196063,-6.354211,2.587334,2.506730,3.311198,-0.104480,3.238238,10.964034,16,...,4.507524,10.078007,28,16,3.255544,2.209689,5.150007,4.039338,3.121230,1.975557
6459,3,8.073248,-4.582502,2.830745,3.447650,3.302172,-0.492919,-0.453013,10.904337,11,...,3.758635,20.366468,27,29,3.246669,2.650009,4.630977,3.314692,4.437069,2.838800
6460,3,9.035717,-6.904877,-0.025219,-0.399831,3.414785,0.284005,0.775833,11.660759,1,...,2.128289,19.916885,1,29,3.357390,2.592356,5.177956,3.986420,4.387823,2.956030
6461,3,8.398860,-6.224924,0.964544,1.376666,2.958615,-0.059084,0.721621,8.753403,20,...,4.863290,39.297190,19,23,2.908887,2.250614,4.252132,3.073357,6.163382,4.058800


In [152]:
classification_human_activity

{'walking': 0, 'running': 1, 'idle': 2, 'stairs': 3}

## Random Forest

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [159]:
X = df.iloc[:, 1:]
y = df.iloc[:, 0]